In [24]:
def compute_recall(groundtruth, detection, score, overlap_ratio):
    #print("==================")
    tp_idx = []   
    for idx_gt, gt in enumerate(groundtruth):
        #print("gt={}".format(gt))
        area = (gt[3] - gt[1]) * (gt[2] - gt[0])
        for idx, dt in enumerate(detection):
            #print("dt = {}".format(dt))
            x2 = min(max(dt[0], dt[2]), max(gt[0], gt[2]))
            y2 = min(max(dt[1], dt[3]), max(gt[1], gt[3]))
            x1 = max(min(dt[0], dt[2]), min(gt[0], dt[2]))
            y1 = max(min(dt[1], dt[3]), min(gt[1], gt[3]))
            #print("y_diff = {}".format(y2-y1))
            #print("x_diff = {}".format(x2-x1))
            if y2 > y1 and x2 > x1:
                overlap = float((y2-y1)*(x2-x1)) / float(area)
                #print("overlap = {}".format(overlap))
                #print("overlap_ratio = {}".format(overlap_ratio))
                #print("dt[4] = {}".format(dt[4]))
                #print("score = {}".format(score))
                if (overlap > overlap_ratio and dt[4] - score > 0):
                    tp_idx.append(idx)
                    break
    
    #print("---tp = {}".format(len(tp_idx)))
    #print("---gt = {}".format(len(groundtruth)))
    recall = float(len(tp_idx)) / float(len(groundtruth))
    return recall, tp_idx

def compute_pos(detection, score):
    tp_np_idx = []
    for idx, dt in enumerate(detection):
        if (dt[4] > score):
            tp_np_idx.append(idx)
    return tp_np_idx
            

#----------- get classification result -------------#
file=open("res_list_github_with_rotation_new_70000.txt")
#file=open("res_list_github_with_rotation_8_17.txt")
context = file.read()
lines = context.splitlines()

class_dict = {}
for idx, line in enumerate(lines):
    result = line.split(" ")[0]
    score = float(line.split(" ")[1])
    
    name = result.split("/")[3]
    box = result.split("/")[4]
    
    y = int((box.split(".")[0]).split("_")[0])
    x = int((box.split(".")[0]).split("_")[1])
    
    bbox = [x, y, x+512, y+512, score]
    #print("name = {}".format(name))
    #print("--={}".format(class_dict.has_key(name)))
    
    if name in class_dict:
        class_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        class_dict[name] = cur_list
#print(class_dict["2017-06-15_16_09_39"])

#-------------get gt info-------------#
gt_dict = {}
file=open("test_roi_info.txt")
context = file.read()
lines = context.splitlines()
for idx, line in enumerate(lines):
    result = line.split("/")[-1].split(" ")
    if len(result[0].split("_")) > 1:
        name = result[0].split("_")[1] + "_" + result[1].split(".")[0]
    else:
        name = result[0] + "_" + result[1].split(".")[0]
            
    bbox = [int(result[2].split(".")[0]), int(result[3].split(".")[0]), 
            int(result[4].split(".")[0]), int(result[5].split(".")[0])]
    #print(result[2])
    #print(name)
    #print(bbox)
    
    if name in gt_dict:
        gt_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        gt_dict[name] = cur_list
#print(gt_dict["2017-06-15_16_09_39"])    
    
#-------------set thresholds-------------#
def get_thresh(step):
    num = 0
    thresh = []
    while num < 1:
        thresh.append(num)
        num += step
    return thresh

thresh_score = get_thresh(0.05)
thresh_overlap_ratio = get_thresh(0.05)
thresh_num_ratio = get_thresh(0.05)
thresh_score = [0.5]
thresh_overlap_ratio = [0]
thresh_num_ratio = [0.01]

#-------------compute result-------------#
total_neg_slice = len(class_dict) - len(gt_dict)
total_pos_slice = len(gt_dict)
print("total_neg_slice = {}".format(total_neg_slice))
print("total_neg_slice = {}".format(total_neg_slice))

dict_res = []
res_bbox_recall = []
for idx_score, score in enumerate(thresh_score):
    for idx_overlap_ratio, overlap_ratio in enumerate(thresh_overlap_ratio):
        for idx_num_ratio, num_ratio in enumerate(thresh_num_ratio):
        
            bbox_recall = []
            pos_slice_idx = []
            num_dt_neg_slice = 0
            num_tp_pos_slice = 0
            num_tp_neg_slice = 0
            for key in class_dict.keys():
                
                #for bbox_idx, bbox in enumerate class_dict["key"]:
                #if key != "2017-06-15_16_09_39":
                #    continue;
                
                if key in gt_dict:
                    recall, tp_list = compute_recall(gt_dict[key], class_dict[key], score, overlap_ratio)
                    
                else:
                    recall = 0
                    tp_list = []
                bbox_recall.append([key, recall])
                print("--------------------")
                print("key = ", key)
                print("recall = {}".format(recall))
                print("pos = {}".format(tp_list))
                
                tp_np_list = compute_pos(class_dict[key], score)
                positive_ratio = float(len(tp_np_list)) / float(len(class_dict[key]))
                print("pos = {}".format(len(tp_np_list)))
                print("total = {}".format(len(class_dict[key])))
                print("positive_ratio = {}".format(positive_ratio))
                if positive_ratio - num_ratio < 0:
                    num_neg_slice += 1 
                    if key not in gt_dict:
                        num_tp_neg_slice += 1
                else:
                    pos_slice_idx.append(key)
                    if key in gt_dict:
                        num_tp_pos_slice += 1
            
            print("num_neg_slice = {}".format(num_neg_slice))
            print("num_tp_neg_slice = {}".format(num_tp_neg_slice))
            
            pos_slice_recall_ratio = float(num_tp_pos_slice) / float(total_pos_slice)
            neg_slice_exclude_ratio = float(num_tp_neg_slice) / float(total_neg_slice)
            res = [score, overlap_ratio, num_ratio, pos_slice_recall_ratio, neg_slice_exclude_ratio, pos_slice_idx]
            res_bbox_recall.append(bbox_recall)
            dict_res.append(res)
            print(dict_res)
            
            print("--------------------------")
            print(res_bbox_recall)
            
                
                
            

total_neg_slice = 10
total_neg_slice = 10
--------------------
key =  2017-06-30_10_20_21
recall = 0
pos = []
pos = 0
total = 3906
positive_ratio = 0.0
--------------------
key =  2018-04-20_14_41_17
recall = 0
pos = []
pos = 0
total = 4221
positive_ratio = 0.0
--------------------
key =  2017-06-15_15_56_02
recall = 0.0
pos = []
pos = 6
total = 3906
positive_ratio = 0.0015360983102918587
--------------------
key =  2017-06-15_16_09_39
recall = 0.0
pos = []
pos = 13
total = 3780
positive_ratio = 0.0034391534391534392
--------------------
key =  2018-01-09_19_12_32
recall = 1.0
pos = [5177]
pos = 1629
total = 10384
positive_ratio = 0.15687596302003082
--------------------
key =  2018-01-09_20_11_10
recall = 0.75
pos = [7470, 2264, 5800]
pos = 2795
total = 8976
positive_ratio = 0.3113859180035651
--------------------
key =  2018-04-19_15_27_41
recall = 0
pos = []
pos = 3
total = 10944
positive_ratio = 0.00027412280701754384
--------------------
key =  2018-04-20_12_26_31
recall = 0
pos =

!history
history
history


In [ ]:
Z